In [1]:
proj_list = [
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'lcms',
    'libpng', 'libssh',
    'libxml2', 'pcre2',
    'proj4', 're2',
    'sqlite3', 'vorbis',
    'woff2', 'wpantund'
]

In [2]:
target_project = 0

In [3]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [4]:
gc.collect()
torch.cuda.empty_cache()

print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:
prefix_np, postfix_np, label_np, label_len_np = data.getTrainData(proj_list, proj_list[target_project])

Getting data for "boringssl" from "c-ares"
Getting data for "boringssl" from "freetype2"
Getting data for "boringssl" from "guetzli"
Getting data for "boringssl" from "harfbuzz"
Getting data for "boringssl" from "lcms"
Getting data for "boringssl" from "libpng"
Getting data for "boringssl" from "libssh"
Getting data for "boringssl" from "libxml2"
Getting data for "boringssl" from "pcre2"
Getting data for "boringssl" from "proj4"
Getting data for "boringssl" from "re2"
Getting data for "boringssl" from "sqlite3"
Getting data for "boringssl" from "vorbis"
Getting data for "boringssl" from "woff2"
Getting data for "boringssl" from "wpantund"


In [6]:
test_prefix, test_postfix, test_label, test_label_len = data.getTestData(proj_list[target_project])

In [7]:
train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
    prefix_np, postfix_np, label_np, test_size = 0.2, random_state = 43
)

In [8]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, train_postfix,
        val_prefix, val_postfix,
        test_prefix, test_postfix,
        train_label, val_label, test_label,
        batch_size=1000
    )

In [9]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/developing/tests')

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [11]:
# ====================
# set parameters here
# ====================

title = proj_list[target_project] + '_developing1'
epochs = 1

max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.0
weight_decay = 0.0

embed_dim = 25
hidden_size = 50
n_layers = 1
output_size = max(token_choices) + 2
dropout = 0.0
max_length = max_len
input_size = max(token_choices) + 2
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "CEL"

teacher_forcing = True

In [12]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device,
    model_name='seq2seq',
    optim_name='Adam',
    loss_fn_name='CEL',
    teacher_forcing=teacher_forcing
)

In [ ]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

end_time = (timeit.default_timer() - start_time)

In [ ]:
mu.saveModel(title, model)

In [13]:
model = mu.getModel(title)
print(model)

MySeq2Seq(
  (loss_fn): CrossEntropyLoss()
  (prefixEncoder): Encoder(
    (embedding): Embedding(216, 25)
    (lstm): LSTM(25, 50, batch_first=True, bidirectional=True)
  )
  (postfixEncoder): Encoder(
    (embedding): Embedding(216, 25)
    (lstm): LSTM(25, 50, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(216, 25)
    (lstm): LSTM(25, 50, batch_first=True, bidirectional=True)
  )
  (attn): Attention(
    (fc): Linear(in_features=400, out_features=216, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
)


In [14]:
# loss, acc = tester.test(
#     test_dataloader=test_dataloader,
#     device=device,
#     model=model,
#     title=title
# )

In [15]:
# with open('../stat/developing1', 'a') as f:
#         text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
#         f.write(text)

In [16]:
mu.graphModel(train_dataloader, model, writer)

Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)
Error occurs, No graph saved


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)